In [166]:
library(tidyverse)

## Loggerdaten Aufbereiten

In [219]:
# defining a function to convert the time format from 12h to 24h
change_to_24 = Vectorize(function(x) {
    gsub("\\s+", "", x) -> x
    if (grepl("AM", x)) {
        substr(x, 1, nchar(x) - 3) -> new
        strsplit(new, ":")[[1]] -> temp
        if (temp[1] == "12") {
            temp[1] <- "00"
        }
        if (nchar(temp[1]) == 1) {
            temp[1] <- paste0("0", temp[1])
        }
        paste(temp, collapse = ":") %>% as.character() -> out
    }
    else
    {
        substr(x, 1, nchar(x) - 3) -> new
        strsplit(new, ":")[[1]] -> temp
        if (temp[1] != "12") {
            temp[1] <- as.numeric(temp[1]) + 12
        }
        if (temp[1] == "12") {
            temp[1] <- "12"
        }
        paste(temp, collapse = ":") %>% as.character() -> out
    }

    return(out)
}
)

# define function to ad leading zeros to the date
add_zeros = function(x) {
    if (nchar(x) == 1) {
        paste0("0", x) -> out
    }
    else
    {
        out <- x
    }
    return(out)
}


# defining a function to convert the date format
date_conv = Vectorize(function(x) {
    strsplit(x, "/")[[1]] -> temp
    add_zeros(temp[1]) -> mm
    add_zeros(temp[2]) -> dd
    paste0("20", temp[3]) -> yyyy
    paste(yyyy, mm, dd, sep = "-") -> out
    return(out)
}
)

# defining a function to import and convert the data
import_conv = function(source, name) {
    read.csv(source, skip = 14) %>% 
    as_tibble() %>%
    mutate(
        time = paste(date_conv(Date), change_to_24(Time)) %>% as_datetime()
    ) %>% select(time, !!sym(name) := Value) -> out
    return(out)
}

# defining a function to import and merge all the files
import_merge = function(source, name) {
    full_join(import_conv(source[1], name[1]), import_conv(source[2], name[2]), by = "time") %>%
                 full_join(import_conv(source[3], name[3]), by = "time") -> out
    return(out)
}

In [217]:
c("./data/raw/logger/LoggerGruppe3ober.csv", "./data/raw/logger/LoggerGruppe3unten1.csv","./data/raw/logger/LoggerGruppe3unten2.csv")-> source
c("ober", "unter1", "unter2") -> name

import_merge(source, name) %>% write.csv("./data/logger_winter.csv", row.names = FALSE)